In [20]:
import geopandas as gpd
import pandas as pd
from datetime import datetime
import json
import os

In [21]:
# Prevent warning messages from openpyxl
# Courtesy of https://stackoverflow.com/a/66015981/4361039

import warnings
warnings.simplefilter("ignore")

In [22]:
# Function to read census tract boundaries geopackage file
# Returns a geodataframe
def read_geo(year):
    geo = gpd.read_file(
        f'./census-tracts/{year}-toronto-ct.gpkg'
    ).to_crs('EPSG:4326')
    
    # Identify column with census tract names
    ct_col_name = [ col for col in geo.columns if str(col).lower()[:2] == 'ct'][0]
    if year == 2011:
        ct_col_name = 'CTNAME'
    
    # Only return two columns: ct, geometry
    geo = geo.filter([ct_col_name, 'geometry'])
    geo.columns = ['ct', 'geometry']
    
    # Make sure census tract names are strings, not ints
    geo.ct = geo.ct.apply(lambda x: "{:06.2f}".format(float(x)))
    
    return geo


# Function to read a single year of Census data from Excel
# Returns a dataframe
def read_data(year):
    
    dfs = pd.read_excel(
        f'./census-data/QC Completed {year}.xlsx',
        sheet_name=None,        # read all available sheets
        na_values=['no data']
    )
    
    # Sheets that contain census tract data are named "CT Tab..."
    data_tab_names = [ tab for tab in dfs if tab.startswith('CT Tab') ]
    
    print(f"Census {year}: reading data from {', '.join(data_tab_names)}")
    
    # Combine data across tabs into a single df
    data = pd.concat( [ dfs[tab] for tab in data_tab_names ] )
    
    # Lowercase column names
    data.columns = [ str(x).lower().strip() for x in data.columns ]
    data = data.rename(columns={
        'ct name': 'ct'
    })
    
    # Make sure census tracts are strings, not ints
    data.ct = data.ct.apply(lambda x: "{:06.2f}".format(float(x)))
    
    return data


# Reads a single year of census
# Returns a geodataframe
def read_census(year):
    
    data = read_data(year)
    geo = read_geo(year)
    
    return geo.merge(data, how='right', on='ct')

## Read all Census years

In [23]:
#c1961 = read_census(1961)
c2001 = read_census(2001)

Census 2001: reading data from CT Tab - F-Etobicoke


## Clean up and derive extra variables

In [24]:
# Create a new column with % value for a given numerator
# and denominator, & custom suffix
# Place it after straight after the existing variable
def calc_perc(df, num, denom, suffix='%'):
    col_idx = df.columns.get_loc(num)
    try:
        df.insert(
            col_idx + 1,
            f"{num} (%)",
            (df[num] / (df[denom] if type(denom) == str else denom) * 100).apply(lambda x: round(x, 1))
        )
    except:
        print(f"Could not create a % variable for {num}")
        
        
def calc_sum(df, new_var, existing_vars):
    try:
        df.insert(
            len(df.columns),
            new_var,
            df[existing_vars].sum(axis=1)
        )
    except:
        print(f"Could not create a sum variable {new_var}")

## Read existing and derive new variables for 1951

In [25]:
# Read 1951 census
c1951 = read_census(1951)
    
# Calculate %s for origin
for x in [
    '22 british isles origine',
    '23 french',
    '24 german',
    '25 italian',
    '26 jewish',
    '27 netherlands',
    '28 polish',
    '29 russian',
    '30 scandinavian',
    '31 ukraininian',
    '32 other european',
    '33 asiatic',
    '34 other and not stated',
    '46 roman catholic'
]:
    calc_perc(c1951, x, '1 population, 1951')

    
# Calculate %s for dwellings
for x in [
    '12 [households] with lodgers',
    '29 [occupied dwellings] single detached',
    '30 [occupied dwellings] apartments and flats',
    '33 [occupied dwellings] owner occupied',
    '35 [occupied dwellings] tenant-occupied',
    '37 [year of occupancy:] before 1946',
    '38 [year of occupancy:] 1946-1949', '39 [year of occupancy:] 1950-51',
    '50 [occupied dwellings] passenger automobile'
]:
    calc_perc(c1951, x, '1 occupied dwellings (households)')

# Mortgage needs a different denominator    
calc_perc(
    c1951,
    '34 [occupied dwellings, owner occupied] reporting a mortgage',
    '33 [occupied dwellings] owner occupied',
    suffix='(% own. occup.)'
)


# Earnings - calculate total (M+F)
calc_sum(
    c1951, "[total] earnings - under $1,000", [
        '41 [earnings of wage-earners [males] under $1,000',
        '47 [earnings of wage-earners [females] under $1,000'
    ]
)

calc_sum(
    c1951, "[total] earnings - $1,000-$1,999", [
        '42 [earnings of wage-earners [males] $1,000-$1,999',
        '48 [earnings of wage-earners [females] $1,000-$1,999'
    ]
)

calc_sum(
    c1951, "[total] earnings - $2,000-$2,999", [
        '43 [earnings of wage-earners [males] $2,000-$2,999',
        '49 [earnings of wage-earners [females] $2,000-$2,999'
    ]
)

calc_sum(
    c1951, "[total] earnings - $3,000-$3,999", [
        '44 [earnings of wage-earners [males] $3,000-$3,999',
        '50 [earnings of wage-earners [females] $3,000-$3,999'
    ]
)

calc_sum(
    c1951, "[total] earnings - $4,000 and over", [
        '45 [earnings of wage-earners [males] $4,000 and over',
        '51 [earnings of wage-earners [females] $4,000 and over'
    ]
)


# Calculate % earnings for males
earnings_m_denom = c1951.loc[:,
    '41 [earnings of wage-earners [males] under $1,000':'45 [earnings of wage-earners [males] $4,000 and over'
].sum(axis=1)

for x in [
    '41 [earnings of wage-earners [males] under $1,000',
    '42 [earnings of wage-earners [males] $1,000-$1,999',
    '43 [earnings of wage-earners [males] $2,000-$2,999',
    '44 [earnings of wage-earners [males] $3,000-$3,999',
    '45 [earnings of wage-earners [males] $4,000 and over'
]:
    calc_perc(c1951, x, earnings_m_denom)
    

# Calculate % earnings for females
earnings_f_denom = c1951.loc[:,
    '47 [earnings of wage-earners [females] under $1,000':'51 [earnings of wage-earners [females] $4,000 and over'
].sum(axis=1)

for x in [
    '47 [earnings of wage-earners [females] under $1,000',
    '48 [earnings of wage-earners [females] $1,000-$1,999',
    '49 [earnings of wage-earners [females] $2,000-$2,999',
    '50 [earnings of wage-earners [females] $3,000-$3,999',
    '51 [earnings of wage-earners [females] $4,000 and over'
]:
    calc_perc(c1951, x, earnings_f_denom)
    
# Calculate % earnings for total
earnings_t_denom = c1951.loc[:,
    '[total] earnings - under $1,000':'[total] earnings - $4,000 and over'
].sum(axis=1)

for x in [
    '[total] earnings - under $1,000',
    '[total] earnings - $1,000-$1,999',
    '[total] earnings - $2,000-$2,999',
    '[total] earnings - $3,000-$3,999',
    '[total] earnings - $4,000 and over'
]:
    calc_perc(c1951, x, earnings_t_denom)

Census 1951: reading data from CT Tab-Etobicoke Twshp, CT Tab-Long Branch, CT Tab-New Toronto, CT Tab-Mimico


## 1961

In [36]:
c1961 = read_census(1961)

# Calculate totals for each category broken down by male/female
male_cols = [x for x in list(c1961.columns) if x.startswith('males:')]

for m in male_cols:
    calc_sum(
        c1961,
        'total' + m[5:],
        [m, 'fe' + m]
    )


# Calculate %s for origin & immigration
idx_from = c1961.columns.get_loc('total: religion roman catholic')
idx_to = c1961.columns.get_loc('total: period of immigration: 1961')
cols = list( c1961.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1961, col, 'total: total count')
    
    
# Calculate %s for dwelling types
idx_from = c1961.columns.get_loc('type of dwelling: single detached')
idx_to = c1961.columns.get_loc('mortgage holder: residual (no mortgage)')
cols = list( c1961.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1961, col, 'total dwellings')
    
# TODO: %s for wages

Census 1961: reading data from CT Tab - Etobicoke Twshp, CT Tab - Long Branch, CT Tab - New Toronto, CT Tab - Mimico


## Read existing and derive new variables for 1971

In [27]:
c1971 = read_census(1971)
    
# Calculate %s for origin/birthplace/period of immigration
idx_from = c1971.columns.get_loc('religion: roman catholic')
idx_to = c1971.columns.get_loc('period of immigration: 1965-1971')
cols = list( c1971.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1971, col, 'total population')
    
# Dwellings
calc_perc(c1971, 'total owned dwellings', 'total dwellings')
calc_perc(c1971, 'total rented dwellings', 'total dwellings')


# Dwellings - owned
for x in [
    'owned dwelling: single detached',
    'owned dwelling: single attached',
    'owned dwelling: apartment',
    'owned dwelling: mobile'
]:
    calc_perc(c1971, x, 'total owned dwellings')


# Dwellings - rented
for x in [
    'rented dwelling: single detached',
    'rented dwelling: single attached',
    'rented dwelling: apartment',
    'rented dwelling: mobile'
]:
    calc_perc(c1971, x, 'total rented dwellings')


# Dwellings: period of construction, occupancy, automobile
idx_from = c1971.columns.get_loc('dwellings: period of construction: before 1946')
idx_to = c1971.columns.get_loc('dwellings: length of occupancy: more than 10 years')
cols = list( c1971.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1971, col, 'total dwellings')
    
    
# Household income - %s
hh_income_denom = c1971.loc[:,
    'households: income from all sources: under $3,000' : 'households: income from all sources: $15,000 & over'
].sum(axis=1)

for x in [
    'households: income from all sources: under $3,000',
    'households: income from all sources: $3,000-$4,999',
    'households: income from all sources: $5,000-$6,999',
    'households: income from all sources: $7,000-$9,999',
    'households: income from all sources: $10,000-$14,999',
    'households: income from all sources: $15,000 & over'
]:
    calc_perc(c1971, x, hh_income_denom)

Census 1971: reading data from CT Tab - Borough of Etobicoke


## 1981

In [28]:
c1981 = read_census(1981)
    
# Calculate %s for origin/birthplace/period of immigration
idx_from = c1981.columns.get_loc('religion - catholic')
idx_to = c1981.columns.get_loc('period of immigration(7)(*) - 1979 - 1981')
cols = list( c1981.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1981, col, 'population, 1981')

    

# Dwellings - types & period of construction
idx_from = c1981.columns.get_loc('occupied private dwellings - owned')
idx_to = c1981.columns.get_loc('occupied private dwellings: period of construction - 1971 - 1981(8)')
cols = list( c1981.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1981, col, 'occupied private dwellings, total')


    
# Household Income
for x in [
    'private household income - all households- under $5,000',
    'private household income - all households- $ 5,000-$ 9,999',
    'private household income - all households- $10,000-$14,999',
    'private household income - all households- $15,000-$19,999',
    'private household income - all households- $20,000-$24,999',
    'private household income - all households- $25,000-$29,999',
    'private household income - all households- $30,000-$39,000',
    'private household income - all households- $40,000 & over'
]:
    calc_perc(c1981, x, 'private household income - all households')

Census 1981: reading data from CT Tab - Borough of Etobicoke


## 1991

In [29]:
c1991 = read_census(1991)
c1991.drop('total number of occupied private dwellings.1', axis=1, inplace=True)

# Religion
for x in [
    'religion, catholic, total',
    'religion, catholic, roman catholic',
    'religion, catholic, ukrainian catholic',
    'other religions - jewish'
]:
    calc_perc(c1991, x, 'population, 1991')
    
    
# Single ethnic origin: %s
idx_from = c1991.columns.get_loc('french origins, ethnic origin')
idx_to = c1991.columns.get_loc('other single origins, ethnic origin')
cols = list( c1991.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1991, col, 'ethnic origin, single origins')


# Immigrant population
for x in [
    'non-immigrant population, total',
    'immigrant population',
    '1961-1970, period of immigration',
    '1971-1980, period of immigration',
    '1981-1991, period of immigration',
    '1981-1987, period of immigration',
    '1988-1991, period of immigration'
]:
    calc_perc(c1991, x, 'population, 1991')

    
# Dwellings by type: %
idx_from = c1991.columns.get_loc('occupied private dwellings - owned')
idx_to = c1991.columns.get_loc('occupied private dwellings - movable dwellin')
cols = list( c1991.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1991, col, 'total number of occupied private dwellings')
    
    
# Dwellings by age
for x in [
    'before 1946, period of construction',
    '1946 - 1960, period of construction',
    '1961 - 1970, period of construction',
    '1971 - 1980, period of construction',
    '1981 - 1985, period of construction',
    '1986 - 1991, period of construction (27)'
]:
    calc_perc(c1991, x, 'total number of occupied private dwellings')


# Household income by band, %
idx_from = c1991.columns.get_loc('under $10,000, household income')
idx_to = c1991.columns.get_loc('$70,000 and over, household income')
cols = list( c1991.columns[idx_from : idx_to+1] )

for col in cols:
    calc_perc(c1991, col, 'household income - all private households')

Census 1991: reading data from CT Tab - City of Etobicoke


## 2011

In [30]:
c2011 = read_census(2011)

universe = 'household and dwelling characteristics / total number of occupied private dwellings by structural type of dwelling'

for x in [
    universe + ' / single-detached house',
    universe + ' / apartment, building that has five or more storeys',
    universe + ' / movable dwelling',
    universe + ' / other dwelling / semi-detached house',
    universe + ' / other dwelling / row house',
    universe + ' / other dwelling / apartment, duplex',
    universe + ' / other dwelling / apartment, building that has fewer than five storeys',
    universe + ' / other dwelling / other single-attached house'
]:
    calc_perc(c2011, x,  universe)

Census 2011: reading data from CT Tabs F-Etobicoke


## Save map data

In [31]:
# Dictionary of census years and data frames
dfs = { 1951: c1951, 1961: c1961, 1971: c1971, 1981: c1981, 1991: c1991, 2001: c2001, 2011: c2011 }

# Generate a JSON object with year->variable->min/max values for choropleth
data = {
    str(y) : {
        str(v): {
            'min': str(dfs[y][v].min()),
            'max': str(dfs[y][v].max()),
            'median': str(dfs[y][v].median()),
            'na': str(dfs[y][v].isna().sum())
        } for v in dfs[y].columns[2:]
    } for y in dfs.keys()
}

os.system(f"echo 'const metadata = {json.dumps(data)}' > ./map/metadata.js")

for y, df in zip(dfs.keys(), dfs.values()):
    df.to_file(f'./map/geojson/{y}.geojson', driver='GeoJSON')

## Generate static maps

In [32]:
#fields = list(c1951.columns[2:])

#for field in fields:
#    fig = generate_map(c1951, field, 1951)
#    fig.savefig(f'output/1951-{field}.pdf', dpi=300)